<a href="https://colab.research.google.com/github/OmdenaAI/RebootRx/blob/master/RebootRx/experiments/Nilakshan/hunflair_NER_RCT_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%bash
pip install flair > null


## **Import packages and datafiles**

In [4]:
from google.colab import auth
import pandas as pd
from tqdm import tqdm
import gspread
from oauth2client.client import GoogleCredentials

# setup
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

pd.set_option('display.max_colwidth', 500)
tqdm.pandas()

In [ ]:
!cp  /content/drive/MyDrive/Colab*/DATA/flair_ner/rct_data.csv .

In [5]:
# get squad json
 

reboot_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DATA/flair_ner/rct_data.csv', encoding="utf-8")
reboot_data['Abstract#'] = ['Abstract_{}'.format(i) for i in range(len(reboot_data['Labeled Data']))]

reboot_data = reboot_data[['Abstract#','Labeled Data','Label']]
# reboot_data.set_index(reboot_data['Abstract#'])
reboot_data.head()








,Abstract#,Labeled Data,Label
0,Abstract_0,"PMID: 23403504 TITLE. Randomized double blind trial of ciprofloxacin prophylaxis during induction treatment in childhood acute lymphoblastic leukemia in the WK-ALL protocol in Indonesia. TITLE END DRUG NAME: Ciprofloxacin OBJECTIVES: Toxic death is a big problem in the treatment of childhood acute lymphoblastic leukemia (ALL), especially in low-income countries. Studies of ciprofloxacin as single agent prophylaxis vary widely in success rate. We conducted a double-blind, randomized study to...","{""objects"":[{""featureId"":""ckkc1ibu900033h68m849tw1k"",""schemaId"":""ckjrshew80b7k0y7gaubc7la9"",""title"":""group2"",""value"":""group2"",""color"":""#FF4A46"",""version"":1,""format"":""text.location"",""data"":{""location"":{""start"":723,""end"":733}},""instanceURI"":""https://api.labelbox.com/masks/feature/ckkc1ibu900033h68m849tw1k?token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySWQiOiJja2pzczN6ZDJhejh0MDc2NGlva2oyZmt1Iiwib3JnYW5pemF0aW9uSWQiOiJja2MyZ2h6aDhzY2xxMDc1Nm5rbXA1a3RqIiwiaWF0IjoxNjEyMTEzMjkzLCJleHAiOjE2MTQ3..."
1,Abstract_1,PMID: 15699480 TITLE. Randomized comparison of low molecular weight heparin and coumarin derivatives on the survival of patients with cancer and venous thromboembolism. TITLE END DRUG NAME: Dalteparin PURPOSE: Experimental studies and indirect clinical evidence suggest that low molecular weight heparins may have antineoplastic effects. We investigated the influence of a low molecular weight heparin dalteparin on the survival of patients with active cancer and acute venous thromboembolism. P...,"{""objects"":[{""featureId"":""ckkc1p8do000h3h68jt68ttd7"",""schemaId"":""ckjrshew80b7i0y7g9zzydsj4"",""title"":""group1"",""value"":""group1"",""color"":""#FF34FF"",""version"":1,""format"":""text.location"",""data"":{""location"":{""start"":403,""end"":412}},""instanceURI"":""https://api.labelbox.com/masks/feature/ckkc1p8do000h3h68jt68ttd7?token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySWQiOiJja2pzczN6ZDJhejh0MDc2NGlva2oyZmt1Iiwib3JnYW5pemF0aW9uSWQiOiJja2MyZ2h6aDhzY2xxMDc1Nm5rbXA1a3RqIiwiaWF0IjoxNjEyMTEzMjkzLCJleHAiOjE2MTQ3..."
2,Abstract_2,PMID: 27330919 TITLE. The AVOCAT study: Bicalutamide monotherapy versus combined bicalutamide plus dutasteride therapy for patients with locally advanced or metastatic carcinoma of the prostate-a long-term follow-up comparison and quality of life analysis. TITLE END DRUG NAME: Dutasteride PURPOSE: Compare the efficacy and tolerability of dutasteride in combination with bicalutamide to bicalutamide monotherapy in the treatment of locally advanced and metastatic prostate cancer (PCa). METHODS...,"{""objects"":[{""featureId"":""ckkc1yd23000y3h686shhi3mi"",""schemaId"":""ckjyndgxu07e50zfn7by41vq7"",""title"":""PMID"",""value"":""pmid"",""color"":""#B903AA"",""version"":1,""format"":""text.location"",""data"":{""location"":{""start"":6,""end"":13}},""instanceURI"":""https://api.labelbox.com/masks/feature/ckkc1yd23000y3h686shhi3mi?token=eyJhbGciOiJIUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VySWQiOiJja2pzczN6ZDJhejh0MDc2NGlva2oyZmt1Iiwib3JnYW5pemF0aW9uSWQiOiJja2MyZ2h6aDhzY2xxMDc1Nm5rbXA1a3RqIiwiaWF0IjoxNjEyMTEzMjkzLCJleHAiOjE2MTQ3MDUyOTN..."
3,Abstract_3,"PMID: 1021226 TITLE. Evaluation of tamoxifen dose in advanced breast cancer: a progress report. TITLE END DRUG NAME: Fluoxymesterone The results of an ongoing trial randomizing patients with progressive, metastatic breast carcinoma between tamoxifen (Tam, NSC-180973) and Tam plus fluoxymesterone (Flu) (7 mg/m2 bid) are reported. Each patient received a single dose level of Tam in the range of 2-100 mg/m2 bid. The combination had a higher response rate overall (45% vs 28%) and when only the ...","{""objects"":[{""featureId"":""ckkc2ay3f001l3h68c05rx5hq"",""schemaId"":""ckjyndgxu07e50zfn7by41vq7"",""title"":""PMID"",""value"":""pmid"",""color"":""#B903AA"",""version"":1,""format"":""text.location"",""data"":{""location"":{""start"":6,""end"":12}},""instanceURI"":""https://api.labelbox.c

##**Get the tokens,tags and locations of tags**

In [6]:
data_dict = {'abstract#':[],'abstract':[],'tokens':[],'tags':[],'location':[]}
for i in range(reboot_data.shape[0]):
  abstract_num = reboot_data['Abstract#'][i]
  label= reboot_data['Label'][i]
  abstract = reboot_data['Labeled Data'][i]
  d = eval(label)
  try:
    assert(len(d)>0)
  except:
    continue   
   
   
  token_list = []
  tag_list = []
  loc_list = []

  
  
  for tag_dict in d['objects']:
    
    tag = tag_dict['value']
    location = tag_dict['data']['location']
    
    token = abstract[location['start']:location['end']+1].lower()
    token_list.append(token)
    tag_list.append(tag)
    loc_list.append((location['start'],location['end']))
  
 
  abst_tokens = [i for i in abstract.split()]
  token_l = token_list#+[i for i in abst_tokens if i not in token_list]
  tag_l = tag_list#+['O']*(len(abst_tokens)-len(token_list))
  loc_l = loc_list#+[(len(abst_tokens),len(abst_tokens)+1) for i in abst_tokens if i not in token_list]
   
  data_dict['abstract#'].append(abstract_num)
  data_dict['abstract'].append(abstract)
  data_dict['tokens'].append(token_l)
  data_dict['tags'].append(tag_l)
  data_dict['location'].append(loc_l)

In [7]:
  [i for i in zip(token_l,tag_l)]

[('one hundred, twenty-two', 'total_sample_size'),
 ('buserelin', 'group1'),
 ('buserelin and flutamide', 'group2'),
 ('buserelin', 'group1'),
 ('59 patients', 'g1_n_response'),
 ('61.4%', 'g1_response_rate'),
 ('61.4', 'g2_response_rate'),
 ('59 patients', 'g2_n_response')]

In [8]:
df = pd.DataFrame(data_dict)

In [9]:
#lowecasing the abstract tokens
df['abstract'] = df.abstract.str.lower()
df.head(2)

,abstract#,abstract,tokens,tags,location
0,Abstract_0,"pmid: 23403504 title. randomized double blind trial of ciprofloxacin prophylaxis during induction treatment in childhood acute lymphoblastic leukemia in the wk-all protocol in indonesia. title end drug name: ciprofloxacin objectives: toxic death is a big problem in the treatment of childhood acute lymphoblastic leukemia (all), especially in low-income countries. studies of ciprofloxacin as single agent prophylaxis vary widely in success rate. we conducted a double-blind, randomized study to...","[placebo arm, placebo, 58, 52, ciprofloxacin monotherapy]","[group2, group2, g1_n, g2_n, group1]","[(723, 733), (784, 790), (708, 709), (740, 741), (517, 541)]"
1,Abstract_1,pmid: 15699480 title. randomized comparison of low molecular weight heparin and coumarin derivatives on the survival of patients with cancer and venous thromboembolism. title end drug name: dalteparin purpose: experimental studies and indirect clinical evidence suggest that low molecular weight heparins may have antineoplastic effects. we investigated the influence of a low molecular weight heparin dalteparin on the survival of patients with active cancer and acute venous thromboembolism. p...,"[dalteparin, dalteparin, dalteparin, dalteparin, coumarin, 602, 0.50, 15699480, .02]","[group1, group1, group1, group1, group2, g1_n, pfs_hr, pmid, response_p]","[(403, 412), (900, 909), (1191, 1200), (1522, 1531), (670, 677), (1025, 1027), (1277, 1280), (6, 13), (1646, 1648)]"


In [10]:
sample = df['abstract'].sample(n=1, random_state=42).iloc[0]
sample

'pmid: 15863204  title. adaptive randomized study of idarubicin and cytarabine alone or with interleukin-11 as induction therapy in patients aged 50 or above with acute myeloid leukemia or high-risk myelodysplastic syndromes. title end drug name: oprelvekin a higher complete remission (cr) rate was observed in patients with acute myeloid leukemia (aml) who, on a prior randomized study of induction therapy, received gemtuzumab ozogamicin (go) plus interleukin-11 (il-11) rather than go alone. an adaptive randomized phase iii study of the addition of il-11 to idarubicin and cytarabine (ia) induction in 100 patients >/=50 years of age with aml or high-risk myelodysplastic syndrome (mds) was conducted. median patient age was 67 years (range 50-82). twenty-four of the 45 (53%) patients randomized to ia plus il-11 achieved cr. eight (33%) subsequently relapsed, 4 (17%) died in cr; median time to treatment failure (ttf) was 37 weeks. twenty-nine of the 55 (53%) patients treated without il-11 a

I used this fucntion to tokenize abstract to add 'O' label to uneanted tokens(Not used now) . **Use sci-spacy's tokenization**

In [ ]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize



def  tokenize(text):
  nltk_tokens = word_tokenize(text)
  # convert to lower case
  lower_tokens = [w.lower() for w in nltk_tokens]
  # remove punctuation from each word
   
  # table = str.maketrans('', '', string.punctuation)
  # stripped_tokens = [w.translate(table) for w in lower_tokens]
  # remove remaining tokens that are not alphabetic
  # tokens = [word for word in stripped_tokens if word.isalpha()]
  # filter out stop words
 
  # stop_words = set(stopwords.words('english'))
  # tokens = [w for w in lower_tokens if not w in stop_words]
  tokens = lower_tokens
  return tokens



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def clean_abstract(text):
  tokens = tokenize(text)
  return ' '.join(tokens)

In [ ]:
df.abstract.keys()

RangeIndex(start=0, stop=527, step=1)

In [ ]:
clean_abstract(df.abstract[290])

'pmid : 30017831 title . nedaplatin plus docetaxel versus cisplatin plus docetaxel as first-line chemotherapy for advanced squamous cell carcinoma of the lung - a multicenter , open-label , randomized , phase iii trial . title end drug name : pyrimethamine introduction : this study aimed to compare the efficacy of first-line nedaplatin ( 80 mg/m ( 2 ) ) plus docetaxel ( 75 mg/m ( 2 ) ) ( nd ) versus cisplatin ( 75 mg/m ( 2 ) ) plus docetaxel ( 75 mg/m ( 2 ) ) ( cd ) in patients with advanced squamous cell lung carcinoma . methods : this open-label randomized controlled phase iii trial was performed at 12 hospitals in china . patients with squamous cell lung carcinoma were randomized to four cycles of nd or cd . the primary endpoint was progression-free survival ( pfs ) . secondary endpoints included time to progression , best overall response , and adverse events . results : in the intent-to-treat analysis set ( nd : n = 141 ; cd : n = 139 ) , median pfs was 4.63 months ( 95 % confiden

Created a new dataframe to create a dataset to fit FLAIR model. 
To-Do:
1. Need to check whether alternate format work
2. Plug-in Praful's char--> token labeling

In [11]:

data_df = pd.DataFrame()
data_df['abstract'] = df['abstract']
data_df['annotation']  = [0]*len(df)
tok = df['tokens']
tag = df['tags']
 

In [13]:
for k in tag.keys():
  data_df['annotation'][k] = list(([i for i in zip(tok[k],tag[k])]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
data_df.head(2)

,abstract,annotation
0,"pmid: 23403504 title. randomized double blind trial of ciprofloxacin prophylaxis during induction treatment in childhood acute lymphoblastic leukemia in the wk-all protocol in indonesia. title end drug name: ciprofloxacin objectives: toxic death is a big problem in the treatment of childhood acute lymphoblastic leukemia (all), especially in low-income countries. studies of ciprofloxacin as single agent prophylaxis vary widely in success rate. we conducted a double-blind, randomized study to...","[(placebo arm, group2), (placebo, group2), (58, g1_n), (52, g2_n), (ciprofloxacin monotherapy, group1)]"
1,pmid: 15699480 title. randomized comparison of low molecular weight heparin and coumarin derivatives on the survival of patients with cancer and venous thromboembolism. title end drug name: dalteparin purpose: experimental studies and indirect clinical evidence suggest that low molecular weight heparins may have antineoplastic effects. we investigated the influence of a low molecular weight heparin dalteparin on the survival of patients with active cancer and acute venous thromboembolism. p...,"[(dalteparin, group1), (dalteparin, group1), (dalteparin, group1), (dalteparin, group1), (coumarin, group2), (602, g1_n), (0.50, pfs_hr), (15699480, pmid), (.02, response_p)]"


In [ ]:
# data_df.groupby('annotation').head() #groupby gave error

Data split

In [16]:
# df_train_test = data_df.sample(frac=0.8, random_state=42)
# df_val = data_df[~data_df.index.isin(df_train_test.index)]

df_train_test = data_df
df_train = df_train_test.sample(frac=0.9, random_state=42)
df_test = df_train_test[~df_train_test.index.isin(df_train.index)]



In [17]:
len(df_train),len(df_test),len(data_df)

(474, 53, 527)

## Converting the tags to IOB-format

In [18]:
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
     
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)
        
    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme. 
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'
        
    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)
            
    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.abstract, df.annotation):
            # text = clean(text)
            text_ = text        
            match_list = []
            for i in annotation:
                
                a, text_ = matcher(text, i[0])
                
                try:
                  a[0]
                except:
                  a = [(i[2])]
                
                # print(i[0],i[1],i[2],a)
                match_list.append((a[0][0], a[0][1], i[1]))

            d = mark_sentence(text, match_list)
            vals = list(d.values())
            for i,(k,v) in enumerate(d.items()):
                tag_curr = vals[i].split('-')[-1]
                rel_curr = vals[i].split('-')[0]

                tag_prev = vals[i-1].split('-')[-1]
                rel_prev = vals[i-1].split('-')[0]

                if rel_curr =='I' and rel_prev=='B':

                  if tag_curr!=tag_prev:
                    continue
                  else:
                    f.writelines(k + ' ' + v +'\n')
                elif rel_curr == 'B' :
                  f.writelines(k + ' ' + v +'\n')
                else:
                  continue
            f.writelines('\n')
            


In [19]:
#sample to test IOB tagging

dict_sample = {'abstract':data_df['abstract'][0],'annotation': [data_df.annotation[0]]}
df_sample = pd.DataFrame(dict_sample)
df_sample

,abstract,annotation
0,"pmid: 23403504 title. randomized double blind trial of ciprofloxacin prophylaxis during induction treatment in childhood acute lymphoblastic leukemia in the wk-all protocol in indonesia. title end drug name: ciprofloxacin objectives: toxic death is a big problem in the treatment of childhood acute lymphoblastic leukemia (all), especially in low-income countries. studies of ciprofloxacin as single agent prophylaxis vary widely in success rate. we conducted a double-blind, randomized study to...","[(placebo arm, group2), (placebo, group2), (58, g1_n), (52, g2_n), (ciprofloxacin monotherapy, group1)]"


In [20]:
create_data(df_sample,'sample.txt')

In [21]:
#Run once-only
## path to save the txt file.
train_file = 'train_rct.txt'
# val_file = 'val_rct.txt'
test_file = 'test_rct.txt'
## creating the file.
create_data(df_train,train_file)
# create_data(df_val,val_file)
create_data(df_test,test_file)


In [ ]:
!cp *txt /content/drive/MyDrive/Colab*/DATA/flair_ner/

## **Load the data to Flair column corpus**

In [4]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '.'#'/content/drive/MyDrive/Colab Notebooks/DATA/flair_ner'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train_rct.txt',
                              test_file = 'test_rct.txt')
                              # dev_file = 'val_rct.txt')

2021-02-10 04:33:43,900 Reading data from .
2021-02-10 04:33:43,901 Train: train_rct.txt
2021-02-10 04:33:43,902 Dev: None
2021-02-10 04:33:43,904 Test: test_rct.txt


In [5]:
print(len(corpus.train))
print(corpus.train[0].to_tagged_string('ner'))

416
100 <B-total_sample_size> 45 <B-g1_n> ia <B-group2> 55 <B-g2_n> without <B-group2> 21 <B-g1_median_os> 59 <B-g2_median_os> weeks <I-g2_median_os> 53% <B-g2_n_response> p=0.271 <B-os_p>


##**Training with HUNFLAIR embeddings**

In [6]:
# tag to predict
tag_type = 'ner'
# make tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

In [3]:


## Importing the Embeddings ##
from flair.embeddings import WordEmbeddings
from flair.embeddings import CharacterEmbeddings
from flair.embeddings import StackedEmbeddings
from flair.embeddings import FlairEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.embeddings import FlairEmbeddings

### Initialising embeddings (un-comment to use others) ###
glove_embedding =   WordEmbeddings("pubmed")
#character_embeddings = CharacterEmbeddings()
flair_forward  =  FlairEmbeddings("pubmed-forward")
flair_backward =  FlairEmbeddings("pubmed-backward")
bert_embedding = BertEmbeddings()
# elmo_embedding = ELMoEmbeddings()

stacked_embeddings = StackedEmbeddings( embeddings = [ glove_embedding,
                                                       flair_forward, 
                                                       flair_backward,
                                                      
                                                      #  bert_embedding,
                                                      #  elmo_embedding

                                                      
                                                      ])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated method __init__. (Use 'TransformerWordEmbeddings' for all transformer-based word embeddings) -- Deprecated since version 0.4.5.


Hunflair's pretrained model not work for my dataset. I observed training-loss reduced rapidly,but no improvement in score

In [7]:
from flair.models import SequenceTagger
tagger : SequenceTagger = SequenceTagger(hidden_size=256,
                                       embeddings=stacked_embeddings,
                                       tag_dictionary=tag_dictionary,
                                       tag_type=tag_type,
                                       use_crf=True)
print(tagger)

SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('pubmed')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4296, out_features=4296, bias=True)
  (rnn): LSTM(4296, 256, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=512, out_features=48, bias=True)
  (beta): 1.0
  (weights): None
  (weight_tensor) None
)


In [8]:
from flair.trainers import ModelTrainer
trainer : ModelTrainer = ModelTrainer(tagger, corpus)
trainer.train('resources/taggers/rct-ner-transfer',
              learning_rate=0.1,
              mini_batch_size=50,
              max_epochs=200)

2021-02-10 04:33:44,229 ----------------------------------------------------------------------------------------------------
2021-02-10 04:33:44,230 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('pubmed')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.1, inplace=False)
        (encoder): Embedding(275, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=275, bias=True)
      )
    )
  )
  (word_dropout): WordDropout(p=0.05)
  (locked_dropout): LockedDropout(p=0.5)
  (embedding2nn): Linear(in_features=4296, out_features=4296, bias=True)
  (rnn): LSTM(4296, 256, batch_first=True, bi

{'dev_loss_history': [24.312015533447266,
  21.205509185791016,
  18.89198112487793,
  18.07809066772461,
  16.665199279785156,
  16.224206924438477,
  15.478423118591309,
  15.527262687683105,
  14.93234634399414,
  15.197062492370605,
  14.47718620300293,
  14.014053344726562,
  13.912481307983398,
  13.710119247436523,
  13.64910888671875,
  13.463723182678223,
  13.062200546264648,
  12.96084213256836,
  13.17529582977295,
  12.891627311706543,
  12.73892593383789,
  12.74117660522461,
  12.6237211227417,
  12.653515815734863,
  12.6953125,
  12.53249454498291,
  12.498496055603027,
  12.526749610900879,
  12.474642753601074,
  12.425210952758789,
  12.458793640136719,
  12.504247665405273,
  12.459986686706543,
  12.448343276977539,
  12.447199821472168,
  12.405558586120605,
  12.4685697555542,
  12.4331636428833,
  12.426139831542969,
  12.4635591506958,
  12.374653816223145,
  12.442900657653809,
  12.38601016998291,
  12.388821601867676,
  12.38217830657959,
  12.3910894393920

In [16]:
!cp resources/taggers/rct-ner-transfer  /content/drive/MyDrive/Colab*/DATA/flair_ner/resources/taggers/ -rf

In [10]:
from flair.data import Sentence
from flair.datasets import DataLoader
from flair.models import SequenceTagger
# load the trained model

model = SequenceTagger.load('/content/resources/taggers/rct-ner-transfer/final-model.pt')
result, score = model.evaluate(corpus.test, out_path=f"predictions.txt")

print(result.log_line)

2021-02-10 05:49:38,251 loading file /content/resources/taggers/rct-ner-transfer/final-model.pt
0.5162	0.5255	0.5208


To do
Add percentages for response rates
Add 'patients' for total sample size 

In [11]:
f = open('predictions.txt','r')
pred = f.read().split('\n')
pred = [j.split() for j in pred]
pred

[['abiraterone', 'B-group1', 'B-group1'],
 ['acetate', 'I-group1', 'I-group1'],
 ['aa', 'B-group1', 'B-group1'],
 ['(1,000', 'I-group1', 'I-group1'],
 ['p', 'B-group2', 'B-response_p'],
 ['0.64', 'B-os_hr', 'B-os_hr'],
 ['797', 'B-g1_n', 'B-g1_n'],
 ['398', 'B-g2_n', 'B-g2_n'],
 [],
 ['tetracycline', 'B-group1', 'B-group1'],
 ['bleomycin', 'B-group2', 'B-group1'],
 ['intrapleural', 'B-group1', 'B-group2'],
 ['62', 'B-total_sample_size', 'B-total_sample_size'],
 ['bleomycin-treated', 'B-group2', 'B-group2'],
 [],
 ['tamoxifen', 'B-group2', 'B-group2'],
 ['and', 'I-group2', 'I-group2'],
 ['sixty-two', 'B-total_sample_size', 'B-total_sample_size'],
 ['11%', 'B-g1_response_rate', 'B-g1_response_rate'],
 ['12%', 'B-g2_response_rate', 'B-g2_response_rate'],
 [],
 ['pralatrexate', 'B-group1', 'B-group1'],
 ['34', 'B-g1_n', 'B-total_sample_size'],
 ['60%', 'B-g1_response_rate', 'B-g1_response_rate'],
 ['12.8', 'B-g1_median_pfs', 'B-g1_median_os'],
 ['months', 'I-g1_median_pfs', 'I-g1_median_os